# Segmentación basada en grafos
Universidad de Chile  
Departamento de Ciencias de la Computación  
CC5508 - Procesamiento y Análisis de Imágenes  
Gabriel De La Parra  

## Introducción
Esta tarea se basa en la implementación del paper *[Efficient Graph-Based Image Segmentation](https://cs.brown.edu/~pff/papers/seg-ijcv.pdf)* de *[Pedro F. Felzenzswalb](https://cs.brown.edu/~pff/)*. 

##### Imágen como grafo
En la investigación se expone una técnica de *[segmentación de imagenes](https://en.wikipedia.org/wiki/Image_segmentation)* utilizando un *[grafo no dirigido](https://es.wikipedia.org/wiki/Grafo#Grafo_no_dirigido)* para representar la imagen de entrada:
* Los nodos del grafo representan los pixeles de la imagen. 
* Los arcos del grafo representan la relación entre pixeles vecinos.
* A cada arco se le asocia un peso. De lo anterior, cada peso determina la relación entre pixeles vecinos.

##### Segmentación
La segmentación consiste en generar conjuntos de nodos (o componentes conexas).

Para lo anterior se definen dos métricas:
* *Diferencia interna de una componente conexa*.
* *Diferencia entre dos componentes conexas*. 

Lo anterior pretende agrupar pixeles en regiones que sean similares entre sí y diferentes a otras regiones.

##### Proceso de la segmentación
Para la segmentación se procede de la siguiente manera:
* Con las métricas anteriores, se itera sobre las fronteras de cada componente.
* Sobre dicha frontera se busca evaluar si (los pesos de) la *diferencia entre las componentes* es suficientemente mayor a (los pesos de) la *diferencia interna* (de cada componente).
* A partir de esta evaluación se puede generar una única región o mantener las dos regiones.

Para mejorar este proceso se define un umbral sobre la evaluación de las diferencias.

Lo anterior se realiza para todos los arcos del grafo.

##### Métricas para la segmentación
La *diferencia entre dos componentes* se define como el peso mínimo a lo largo de (los arcos de) la frontera entre los dos grupos de nodos.

Por su parte, *la diferencia interna*, se define como el peso máximo en el MST de la componente.

Un MST (*[minumum spanning tree](https://en.wikipedia.org/wiki/Minimum_spanning_tree)* o *[árbol cobertor mínimo](https://es.wikipedia.org/wiki/%C3%81rbol_recubridor_m%C3%ADnimo)*) es una estructura de datos que permite ordenar todos los nodos de un grafo.

[![Minimum spanning tree](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Minimum_spanning_tree.svg/300px-Minimum_spanning_tree.svg.png)](https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Minimum_spanning_tree.svg/300px-Minimum_spanning_tree.svg.png)

> Un ejemplo de árbol recubridor mínimo. Cada punto representa un vértice, cada arista está etiquetada con su peso, que en este caso equivale a su longitud. - Wikipedia

Una forma de construir un MST (Hint entregado en el enunciado) es por el método de [Kruskal](https://es.wikipedia.org/wiki/Algoritmo_de_Kruskal). Para esto se requiere ocupar un algoritmo eficiente para determinar:

1. Los componentes que conectan una arista (Find).
2. Mezclar o unir dos componentes (Union). 

#### Union-Find
[Union-Find](https://en.wikipedia.org/wiki/Disjoint-set_data_structure) es un algoritmo para estructuras de datos disjuntas. *Una estructura de datos para conjuntos disjuntos, es una estructura de datos que mantiene un conjunto de elementos particionados en un número de conjuntos disjuntos (no se solapan los conjuntos).* 

*Union*: Une dos subconjuntos en uno solo.

*Find*: Determina a cual subconjunto pertenece un elemento. Esta operación puede usarse para verificar si dos elementos están en el mismo conjunto.

La [complejidad del algoritmo](https://en.wikipedia.org/wiki/Disjoint-set_data_structure#Time_Complexity) se diferencia por 4 formas de implementación:
* *Union* con *Union-by-rank*
* *Find* con *path compression*
* Ambas
* Ninguna

*Union-by-rank*, consiste en añadir el árbol más pequeño a la raíz del árbol más grande. Como la profundidad del árbol afecta el tiempo de ejecución del algoritmo, el árbol con menor profundidad es añadido a la raíz del árbol con mayor profundidad, el cual aumenta su profundidad solo si sus profundidades son iguales.

*Find* con *path compression*, es una forma recursiva de aplanar la estructura del árbol. Cada nodo visitado en el camino hacia la raíz se añade directamente a la raíz ya que todos estos nodos la comparten. El árbol resultante es más aplanado, acelerando operaciones futuras no solo en estos elementos sino también en aquellos que referencian a estos.

#### Indicaciones de la tarea
Para esta tarea se solicita implementar el algoritmo de segmentación de Felzenswalb para imagenes a colores (3 canales). Una restricción dada, es que no se permite convertir la imagen a escala de grises. Adicionalmente se solicita implementar esto para distintos espacios de color: RGB y CIE L\*a\*b. Finalmente se solicita ejecutar esto para 10 imágenes distintas y comparar los resultados.

Dado lo anterior, en el Paper se indica como trabajar en este tipo de casos:
* Construir el grafo con conectividad 8.
* Utilizar una función de pesos basada en la intensidad absoluta entre pixeles:

<center>$w(v_i, v_j) = |I(p_i)-I(p_j)|$</center>

donde:

$w(v_i, v_j)$ es la función de pesos sobre los nodos $v_i$ y $v_j$

$I(p_i)$ y $I(p_j)$ es la intensidad de los pixeles $p_i$ y $p_j$

Adicionalmente:

* Se aplica un filtro Gaussiano, con $\sigma$ recomendado de $0.8$, que no produce cambios perceptibles en la imagen.
* Para imágenes en color, se ejecuta el algoritmo 3 veces, una para cada componente del espacio de color y luego se intersectan los 3 conjuntos. Para esto se colocan 2 pixeles vecinos en la misma componente si aparecen en las 3 componentes de color. Alternativamente, se propone ejecutar el algoritmo una sola vez, tomando los pesos como la distancia entre los pixeles en el espacio de color.

La propuesta del autor es mucho más sencilla para trabajar en distintos espacios de color.

In [ ]:
# Helper function to resize Images
#from skimage import io
#from skimage.transform import rescale


#for image in [f for f in os.listdir(os.getcwd()) if f.endswith('.jpg')]:
#    print(image)
#    img = io.imread(image)
#    image_rescaled = rescale(img, 0.5)
#    io.imsave(image.replace('.jpg','_05.jpg'), image_rescaled)

In [1]:
%matplotlib inline

import os
from scipy import ndimage as ndi
from skimage import io, morphology, filters, exposure, measure, segmentation, color, img_as_float
import matplotlib.pylab as plt
import matplotlib
import numpy as np
from ipywidgets import interact, widgets, HBox
from IPython import display
import math
from random import random, randint

#Ajustar el tamaño de las imágenes:
matplotlib.rcParams['figure.figsize'] = (14,12)

In [62]:
selectImageWidget = widgets.ToggleButtons(
    options=[f for f in os.listdir(os.getcwd()) if f.endswith('_05.jpg')],
    description='Image:',
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
)

colorSpaceWidget = widgets.ToggleButtons(
    options=['RGB', 'CIE LAB', 'CIE LUV', 'HSV', 'RGB CIE', 'XYZ'],
    description='Color Space:',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
)

sigmaSliderWidget = widgets.FloatSlider(
    value=1.5,
    min=0,
    max=3,
    step=0.1,
    description='Sigma:',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)

kSliderWidget = widgets.FloatSlider(
    value=0.7,
    min=0,
    max=1,
    step=0.1,
    description='k:',
    continuous_update=False,
    readout=True,
    readout_format='.1f',
)

minSizeSliderWidget = widgets.IntSlider(
    value=100,
    min=1,
    max=500,
    step=1,
    description='Min Size:',
    continuous_update=False,
    readout=True,
)

#######################################################################################

class Edge:
    def __init__(self):
        self.vertex1 = 0
        self.vertex2 = 0
        self.weight = 0

class Element:
    def __init__(self, i):
        self.parent = i
        self.rank = 0
        self.size = 1        
        
class Set:
    def __init__(self, numberOfElements):
        self.elements = [Element(i) for i in range(numberOfElements)]

    # Determine which subset an element x is in.
    def find(self, x):
        y = x
        while y != self.elements[y].parent:
            y = self.elements[y].parent
        self.elements[x].parent = y
        return y

    # Join two subsets x, y into a single one.
    def join(self, x, y):
        # El que tenga el rank mayor queda como parent:
        if self.elements[x].rank > self.elements[y].rank:
            self.elements[y].parent = x
            self.elements[x].size += self.elements[y].size
        else:
            self.elements[x].parent = y
            self.elements[y].size += self.elements[x].size
            if self.elements[x].rank == self.elements[y].rank:
                self.elements[y].rank += 1

    # Get subset size
    def size(self, x):
        return self.elements[x].size

def thresholdFunction(size, k):
    return k / size

def segmentGraph(numberOfVertices, numberOfEdges, edges, k):
    # Sort edges by weight
    edges = sorted(edges, key=lambda edges: edges.weight)

    # Make a disjoint-set forest
    sets = Set(numberOfVertices)

    # Initialize thresholds (size=1)
    threshold = np.zeros(numberOfVertices)
    for i in range(numberOfVertices):
        threshold[i] = thresholdFunction(1, k)

    # For each edge (non-decreasing order)
    for i in range(numberOfEdges):
        # Vertices connected by this edge
        vertex1 = sets.find(edges[i].vertex1)
        vertex2 = sets.find(edges[i].vertex2)
        # If they belong to different components
        if (vertex1 != vertex2):
            # Check if they need to be joined (and do it)
            if (edges[i].weight <= threshold[vertex1]) and (edges[i].weight <= threshold[vertex2]):
                sets.join(vertex1, vertex2)
                vertex1 = sets.find(vertex1)
                threshold[vertex1] = edges[i].weight + thresholdFunction(sets.size(vertex1), k)

    return sets

def euclideanDistance(image, p1x, p1y, p2x, p2y):
    return math.sqrt(math.pow(image[p1y][p1x][0] - image[p2y][p2x][0], 2)
                     + math.pow(image[p1y][p1x][1] - image[p2y][p2x][1], 2)
                     + math.pow(image[p1y][p1x][2] - image[p2y][p2x][2], 2))

def createGraph(image, sigma, k, minimumSize):
    height = image.shape[0]
    width = image.shape[1]

    # Initialize edges
    edges = [Edge() for i in range(width * height * 4)]
    number = 0
    for y in range(height):
        for x in range(width):
            if x < width - 1:
                edges[number].vertex1 = y * width + x
                edges[number].vertex2 = y * width + (x + 1)
                edges[number].weight = euclideanDistance(image, x, y, x + 1, y)
                number += 1

            if y < height - 1:
                edges[number].vertex1 = y * width + x
                edges[number].vertex2 = (y + 1) * width + x
                edges[number].weight = euclideanDistance(image, x, y, x, y + 1)
                number += 1

            if x < width - 1 and y < height - 1:
                edges[number].vertex1 = y * width + x
                edges[number].vertex2 = (y + 1) * width + (x + 1)
                edges[number].weight = euclideanDistance(image, x, y, x + 1, y + 1)
                number += 1

            if x < width - 1 and y > 0:
                edges[number].vertex1 = y * width + x
                edges[number].vertex2 = (y - 1) * width + (x + 1)
                edges[number].weight = euclideanDistance(image, x, y, x + 1, y - 1)
                number += 1
                
    sets = segmentGraph(width * height, number, edges, k)

    # Process components smaller than the minimum size
    for i in range(number):
        a = sets.find(edges[i].vertex1)
        b = sets.find(edges[i].vertex2)
        if a != b and (sets.size(a) < minimumSize or sets.size(b) < minimumSize):
            sets.join(a, b)

    output = np.zeros((height, width, 3))

    # Generate random colors for components
    colors = np.zeros((3, (width * height)))
    for i in range(width * height):
        for j in range(3):
            colors[j][i] = randint(0, 255)

    for y in range(height):
        for x in range(width):
            comp = sets.find(y * width + x)
            for z in range(3):
                output[y][x][z] = colors[z][comp]

    return output

def loadImage(imageFile, colorSpace, sigma, k, minimumSize):
    image = exposure.rescale_intensity(img_as_float(io.imread(imageFile)))
    if colorSpace=="CIE LAB":
        image = exposure.rescale_intensity(color.rgb2lab(image))
    elif colorSpace=="CIE LUV":
        image = exposure.rescale_intensity(color.rgb2luv(image))
    elif colorSpace=="HSV":
        image = exposure.rescale_intensity(color.rgb2hsv(image))
    elif colorSpace=="RGB CIE":
        image = exposure.rescale_intensity(color.rgb2rgbcie(image))
    elif colorSpace=="XYZ":
        image = exposure.rescale_intensity(color.rgb2xyz(image))
    
    image = filters.gaussian(image, sigma, multichannel=True)
    
    image2 = createGraph(image, sigma, k, minimumSize)
    
    #plt.show()
    fig = plt.figure()
    f1 = fig.add_subplot(121)
    f2 = fig.add_subplot(122)
    
    f1.imshow(image, cmap="gray")
    f2.imshow(image2, cmap="gray")
    plt.show()


    
interact(loadImage, imageFile = selectImageWidget, colorSpace = colorSpaceWidget, sigma = sigmaSliderWidget, k= kSliderWidget, minimumSize=minSizeSliderWidget)

<function __main__.loadImage>

In [77]:
# Class Node
class PixelNode:
    def __init__(self, image, x, y):
        self.x = x
        self.y = y
        self.r = int(image[x][y][0])
        self.g = int(image[x][y][1])
        self.b = int(image[x][y][2])
        self.index = y * image.shape[1] + x

    def __str__(self):
        #return "px: " + str(self.x) + " py: " + str(self.y) + " r: " + str(self.r) + " g: "+ str(self.g) + " b: " + str(self.b)
        return "px: " + str(self.x) + " py: " + str(self.y) + " index: " + str(self.index)

In [78]:
# Class Edge
class PixelEdge:
    def __init__(self, pixelNode1, pixelNode2):
        self.node1 = pixelNode1
        self.node2 = pixelNode2
        self.weight = self.pixelDistance(pixelNode1, pixelNode2)

    @classmethod
    def pixelDistance(self, pixelNode1, pixelNode2):
        return math.sqrt(math.pow(pixelNode1.r - pixelNode2.r, 2)
                         + math.pow(pixelNode1.g - pixelNode2.g, 2)
                         + math.pow(pixelNode1.b - pixelNode2.b, 2))

    def __str__(self):
        return "Node1: " + str(self.node1) + " Node2: " + str(self.node2) + " Weight: " + str(self.weight)

In [79]:
# Class Graph
class PixelGraph:
    def __init__(self, image):
        self.nodes = []
        self.edges = []
        self.nodes, self.edges = self.getGraph(image)
    
    @classmethod
    def getGraph(self, image):
        nodes = []
        edges = []
        rows, cols, _ = image.shape
        for row in range(rows):
            for col in range(cols):
                node = PixelNode(image, row, col)
                nodes.append(node)
                
                if row + 1 < rows:
                    edges.append(PixelEdge(node, PixelNode(image, row + 1, col)))

                if col + 1 < cols:
                    edges.append(PixelEdge(node, PixelNode(image, row, col + 1)))

                if row + 1 < rows and col + 1 < cols:
                    edges.append(PixelEdge(node, PixelNode(image, row + 1, col + 1)))

                if row - 1 >= 0 and col + 1 < cols:
                    edges.append(PixelEdge(node, PixelNode(image, row - 1, col + 1)))
        
        return nodes, edges

In [80]:
class SetNode:
    def __init__(self, index, k):
        self.parent = index
        self.rank = 0
        self.size = 1
        self.threshold = self.calcThreshold(k, self.size)
    
    @classmethod
    def calcThreshold(self, k, size):
        return k / size

In [81]:
# Class Disjoint Set with Threshold check
class DisjointSet:
    def __init__(self, nodeCount, thresholdConstant, minSize):
        self.nodes = [SetNode(i, thresholdConstant) for i in range(nodeCount)]
        self.thresholdConstant = thresholdConstant
        self.minSize = minSize

    def Find(self, node):
        rootNode = node
        while rootNode != self.nodes[rootNode].parent:
            rootNode = self.nodes[rootNode].parent
        self.nodes[node].parent = rootNode
        return rootNode

    def ThresholdUnion(self, edge):
        nodeRoot1 = self.Find(edge.node1.index)
        nodeRoot2 = self.Find(edge.node2.index)
        weightThreshold = edge.weight 
        
        if (self.nodes[nodeRoot1].threshold >= weightThreshold) and (self.nodes[nodeRoot2].threshold >= weightThreshold):
            self.Union(nodeRoot1, nodeRoot2)
            nodeRoot1 = self.Find(edge.node1.index)
            self.nodes[nodeRoot1].threshold = weightThreshold + (self.thresholdConstant / self.nodes[nodeRoot1].size)
    
    
    def MinSizeUnion(self, edge):
        nodeRoot1 = self.Find(edge.node1.index)
        nodeRoot2 = self.Find(edge.node2.index)
        minSize = self.minSize

        if self.nodes[nodeRoot1].size < minSize or self.nodes[nodeRoot2].size < minSize:
            self.Union(nodeRoot1, nodeRoot2)        
        
    def Union(self, nodeRoot1, nodeRoot2):
        if(nodeRoot1 == nodeRoot2):
            return
        
        if self.nodes[nodeRoot1].rank > self.nodes[nodeRoot2].rank:
            self.nodes[nodeRoot2].parent = nodeRoot1
            self.nodes[nodeRoot1].size += self.nodes[nodeRoot2].size
            
        elif self.nodes[nodeRoot1].rank < self.nodes[nodeRoot2].rank:
            self.nodes[nodeRoot1].parent = nodeRoot2
            self.nodes[nodeRoot2].size += self.nodes[nodeRoot1].size
            
        else:
            self.nodes[nodeRoot1].parent = nodeRoot2
            self.nodes[nodeRoot2].size += self.nodes[nodeRoot1].size
            self.nodes[nodeRoot2].rank += 1
            
    def printThis(self):
        for i in range(len(self.parent)):
            print("i:",i,"Parent:",self.parent[i],"Rank:",self.rank[i],"Size:",self.size[i],"Threshold:",self.threshold[i])

In [83]:
img = exposure.rescale_intensity(img_as_float(io.imread('photos-1-17_05.jpg')))

img = filters.gaussian(img, 1.5, multichannel=True)

graph = PixelGraph(img)
rows, cols, chan = img.shape
edges = sorted(graph.edges, key=lambda e: e.weight)
print("Nodes:", len(graph.nodes))
print("Edges:", len(graph.edges))

disjointSet = DisjointSet(len(graph.nodes), 0.7, 300)

for i in range(len(edges)):
    disjointSet.ThresholdUnion(edges[i])    

for i in range(len(edges)):
    disjointSet.MinSizeUnion(edges[i])
        



output = np.zeros((rows, cols, chan))
print("Output", output)

# Generate random colors for components
colors = np.zeros((chan, (rows * cols)))
for i in range(rows * cols):
    for j in range(chan):
        colors[j][i] = randint(0, 255)
print("Colors", colors)
        
for y in range(rows):
    for x in range(cols):
        comp = disjointSet.Find(y * cols + x)
        for z in range(chan):
            output[y][x][z] = colors[z][comp]
print("Output", output)

plt.imshow(output, cmap="gray")
plt.show()


IndexError: list index out of range

In [ ]:


def getEdges(nodes, rows, cols):


## Desarrollo

##### Detalles del algoritmo
Para la segmentación se propone el siguiente algoritmo:
1. Ordenar los arcos por sus pesos, de menor a mayor.
2. Definir una segmentación inicial, en la que cada primer nodo es una componente por si sola.
3. Para cada arco:
    * Construir una componente nueva a partir de la componente anterior:
        * Si los dos nodos no pertenecen a la misma componente anterior y el peso entre ellos  es bajo, comparado con la diferencia entre las dos componentes, unir las componentes. Si no, continua.
4. Repetir 3. para todos los arcos del grafo.


#### Aplicaciones de la segmentación (Conc)
La investigación parte con la necesidad de tener algoritmos eficientes para segmentación de imágenes. Algunas de las aplicaciones que se mencionan son los problemas de dificultad media como reconocimiento estéreo, en donde son necesarias definir  *region of support* para las operaciones de correspondencia. Dichas regiones pueden ser encontradas con algoritmos de segmentación. Los problemas de alto nivel también pueden beneficiarse con algoritmos que sirvan para diferenciar el fondo de los objectos y el reconocimiento de las distintas partes.


#### Características deseadas de la segmentación (Conc)
Según el autor, un algoritmo de segmentación debe ser capaz de: 
1. *Capturar regiones o grupos de importancia perceptual que reflejan aspectos globales de la imagen*. Sobre esto, dos problemas principales son poder caracterizar aquello que es importante de la imagen y especificar que es lo que hace una determinada técnica de segmentación.
2. *Ser eficiente, funcionar en tiempo linear sobre el número de pixeles de una imágen*. Los métodos de segmentación deben funcionar a la par con los métodos de detección de borde u otros algoritmos de bajo nivel.


#### Métodos existentes (conc)
En la época en que se escribió esta investigación, se analizaron otras técnicas existentes. El documento explica como los procesos basados en vectores propios son muy lentos para aplicaciones prácticas. Adicionalmente se menciona que otros métodos, si eficientes, fallan en capturar las propiedades globales (no-locales) de la imágen que son perceptiblemente importantes.